In [1]:
from t_prompts import prompt, dedent
from PIL import Image

In [2]:
# Create a test image
img = Image.new('RGB', (100, 100), color='blue')

# Create a medium complexity prompt with nested structure and image
system = prompt(t"You are a helpful AI assistant.")
task = "analyze this image"

p = dedent(t"""
    {system:sys}
    fdgdffggffffffffffffffffffffffggggggggggggggggggggggggggfdgdffggffffffffffffffffffffffggggggggggggggggggggggggggggggggggggggggggggggggggfdgdffggffffffffffffffffffffffggggggggggggggggggfdgdffggffffffffffffffffffffffggggggggggggggggggggggggggfdgdffggffffffffffffffffffffffggggggggggggggggggggggggggggggggggggggggggggggggggfdgdffggffffffffffffffffffffffgggggggggggggggggg    
    Task: {task:t}
    Image: {img:image}

    {'FOO':_: header=BAR}

    List:
    {[prompt(t"- Item {str(i):{i}}") for i in range(5)]}

    {[prompt(t"- Item {str(i):{i}}") for i in range(5)]:inline:sep=, }
    
    Please provide a detailed analysis.
""")

p

StructuredPrompt(keys=['sys', 't', 'image', ...], num_interpolations=6)